In [1]:
import requests
import json
import math
from nba_api.stats.endpoints import *
import pandas
from scipy import stats
from IPython.display import display, HTML, clear_output

betano_url_games = []
team_stats_dict = {}
ALARME_ON = True



def get_content_betano(game_url):
    r = requests.get(game_url)
    content = r.text
    return content


def get_bet_event(game_url):
    content = get_content_betano(game_url)
    token_mark = '<script>window["initial_state"]='
    inx_start = content.find(token_mark) + len(token_mark)
    inx_end = content.find("</script>", inx_start)
    json_content = content[inx_start:inx_end]
    dic_data = json.loads(json_content)
    bet_event = dic_data["data"]["event"]
    return bet_event


def get_teams(event):
    match = event["name"]
    separator = " - "
    team1, team2 = str.split(match, separator)
    return team1, team2


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def get_bet_target(token_class, bet_team, markets):
    winner_markets = find_by_name(token_class, markets)

    if len(winner_markets) > 0:
        winner_market = winner_markets[0]
    else:
        print(f"ERROR: {token_class} not found")
        return

    tracked_team_list = find_by_name(bet_team, winner_market["selections"])
    if len(tracked_team_list) > 0:
        tracked_team_obj = tracked_team_list[0]
    else:
        print(f"ERROR: {bet_team} not found")
        return
    return tracked_team_obj


def has_triggered_bet(tracked_team_obj, bet_value_expected):
    if tracked_team_obj["price"] >= bet_value_expected:
        msg = f'$ {tracked_team_obj["name"]} alcancou {bet_value_expected}: Atual {tracked_team_obj["price"]}'
        print(msg)
        return msg


def is_second_half(event):
    try:
        period = event["liveData"]["periodDescription"]
        if period != "P1" and period != "P2":
            return True
    except:
        return


def can_bet_for_total_points_on_2nd_half(event, t1_stats, t2_stats):
    if is_second_half(event):
        first = event["liveData"]["results"]["firstPeriodScore"]
        try:
            second = event["liveData"]["results"]["secondPeriodScore"]
        except:
            return 
        firstHome, firstAway = first["home"], first["away"]
        secondHome, secondAway = second["home"], second["away"]
        totalFirstHalf = (
            int(firstHome) + int(firstAway) + int(secondHome) + int(secondAway)
        )
        markets = event["markets"]
        total_points = find_by_name("Total de Pontos", markets)
        if len(total_points) == 0:
            return
        less_than_points_obj = total_points[0]["selections"][1]
        handicap = less_than_points_obj["handicap"]
        price = less_than_points_obj["price"]
        projectedPoints = totalFirstHalf * 2
        can_bet = projectedPoints < handicap

        if can_bet :
            media = t1_stats.media_ultima +   t2_stats.media_ultima
            std = t1_stats.std_ultima +   t2_stats.std_ultima
            probabilidade = calcula_probabilidade(handicap, media, std)
            expec = calcula_expectativa(probabilidade, price)
            msg = f"$ JOGAR TOTAL DE PONTOS 2o Tempo:\n{handicap}->{price}/esperado {projectedPoints}, media:{media}, expec:{expec}, prob:{probabilidade}"
            print(msg)
            return msg


class TeamStats:
    def __init__(self, team_name, pontos):
        cut_section = 4
        self.cut_size = len(pontos) // cut_section
        self.name = team_name
        self.media = round(pontos.mean(), 1)
        self.std = round(pontos.std(), 1)
        self.media_ultima = round(pontos[:self.cut_size].mean(), 1)
        self.std_ultima = round(pontos[:self.cut_size].std(), 1)
        _hot_sorted = pontos.sort_values(ascending=False)[:self.cut_size]
        self.hot_mean = round(_hot_sorted.mean(), 1)
        self.hot_std = round(pontos.sort_values(ascending=False)[:self.cut_size].std(), 1)
        _cold_sorted = pontos.sort_values()[:self.cut_size]
        self.cold_mean = round(_cold_sorted.mean(), 1)
        self.cold_std = round(_cold_sorted.std(), 1)
        self.pontos = pontos

    @property
    def hot_media_mean(self):
        union_average = self.union_average(self.hot_mean,self.cut_size,self.media, len(self.pontos))
        return round(union_average,2)
    
    @property
    def hot_media_std(self):
        union_std = self.union_std(self.hot_mean,self.cut_size,self.hot_std, self.media, len(self.pontos), self.std)
        return round(union_std,2)
    
    @property
    def cold_media_mean(self):
        union_average = self.union_average(self.cold_mean,self.cut_size,self.media, len(self.pontos))
        return round(union_average,2)
    
    @property
    def cold_media_std(self):
        union_std = self.union_std(self.cold_mean,self.cut_size,self.cold_std, self.media, len(self.pontos), self.std)
        return round(union_std,2)

    def union_average(self, avg_1, len_avg_1, avg_2, len_avg_2):
        return (len_avg_1*avg_1 + len_avg_2*avg_2)/(len_avg_1+len_avg_2)
    
    def union_std(self, avg_1, len_avg_1, std_1, avg_2, len_avg_2, std_2):
        numerator = len_avg_1*(std_1)**2 + len_avg_2*(std_2)**2 +len_avg_1*len_avg_2*(avg_1-avg_2)**2
        denominator = len_avg_1+ len_avg_2
        return math.sqrt( numerator/denominator) 


    def __str__(self):
        return f"{self.name}:{self.pnts_esperados_baixo()}<-->{self.pnts_esperados()}"


def get_team_stats(team):
    team_alias_id = {"Los Angeles Clippers": "LA Clippers"}
    teams_data = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]
    if team in team_alias_id:
        team = team_alias_id[team]
    team_id_res = teams_data[teams_data["TEAM_NAME"] == team]["TEAM_ID"]
    team_id = team_id_res.values[0]
    pontos = teamgamelogs.TeamGameLogs(
        season_nullable="2023-24", team_id_nullable=team_id
    ).get_data_frames()[0]["PTS"]
    return TeamStats(team, pontos)


def print_json(dic):
    print(json.dumps(dic))


import pandas as pd
from datetime import datetime as dt

ZEBRA_GAME_FILENAME = "zebras_game.csv"


def to_date_str(x):
    return str(x.date()).replace("-", "")


def get_betano_games():
    betano_games_url = "https://br.betano.com/sport/basquete/eua/nba/441g/"
    content = get_content_betano(betano_games_url)
    marca_inicio = '<script type="application/ld+json">'
    indice_inicio = content.find(marca_inicio) + len(marca_inicio)
    indice_fim = content.find("</script>", indice_inicio)
    json_content = content[indice_inicio:indice_fim]
    dic_data = json.loads(json_content)

    return dic_data


def find_zebras_game():
    betano_games = get_betano_games()
    zebra_games = []
    for game in betano_games:
        url = game["url"]
        evento = get_bet_event(url)
        markets = evento["markets"]

        aposta_vencedor = find_by_name("Vencedor", markets)
        if len(aposta_vencedor) == 0:
            continue

        apostas = aposta_vencedor[0]["selections"]
        team1 = apostas[0]
        team2 = apostas[1]
        zebra_threshold = 4.5
        if team1["price"] >= zebra_threshold or team2["price"] >= zebra_threshold:
            # print(f"! ZEBRA: {team1['name']} X {team2['name']} {url}")
            o = {}
            o["startTime"] = evento["startTime"]
            o["name"] = evento["name"]
            o["id"] = int(evento["id"])
            zebra = team1 if team1["price"] >= zebra_threshold else team2
            o["zebra_name"] = zebra["name"]
            o["zebra_price"] = zebra["price"]
            zebra_games.append(o)
    return zebra_games


def save_betano_zebras():
    zebra_games = find_zebras_game()

    startDateCol = "startDate"
    try:
        df_csv = pd.read_csv(ZEBRA_GAME_FILENAME, index_col="startDate")
    except:
        df_csv = pd.DataFrame(
            columns=[
                startDateCol,
                "startTime",
                "name",
                "id",
                "zebra_name",
                "zebra_price",
            ]
        )
        df_csv = df_csv.set_index("startDate")
    df = pandas.DataFrame()
    df2 = df.from_records(zebra_games)
    df_to_date = lambda x: dt.fromtimestamp(x / 1000)
    df2["startDate"] = [to_date_str(df_to_date(i)) for i in df2["startTime"].values]
    df2 = df2.set_index("startDate")

    ids_existent = df_csv.id.values
    df_csv_news = df2[~df2["id"].isin(ids_existent)]

    df_csv_save = pd.concat([df_csv, df_csv_news])
    df_csv_save.to_csv(ZEBRA_GAME_FILENAME)


def get_zebra_from_today():
    df_csv = pd.read_csv(ZEBRA_GAME_FILENAME)
    today = dt.today()
    today_inx = int(to_date_str(today))
    # df_zebra_today =  df_csv[df_csv['startDate']==today_inx]
    df_zebra_today = df_csv
    return df_zebra_today


save_betano_zebras()


def bet_pnts_by_game(t1_stats: TeamStats, t2_stats: TeamStats, markets):
    total_de_pontos = find_by_name("Total de Pontos", markets)
    if len(total_de_pontos) == 0:
        return None

    menos_de_total_de_pontos = total_de_pontos[0]["selections"][1]
    aposta_tot_pnts = menos_de_total_de_pontos["handicap"]
    maior_pnts_do_jogo_provavel = round(t1_stats.hot_media_mean + t2_stats.hot_media_mean,2)
    menor_pnts_do_jogo_provavel = round(t1_stats.cold_media_mean + t2_stats.cold_media_mean,2)
    # tot_pnts_str = f"(!) TOT_PNTS: {math.floor(menor_pnts_do_jogo_provavel)}<-->{math.floor(maior_pnts_do_jogo_provavel)}"
    # print(tot_pnts_str)

    # if aposta_tot_pnts > maior_pnts_do_jogo_provavel:
    #     preco = menos_de_total_de_pontos["price"]
    #     std_teams = math.sqrt(t1_stats.hot_media_std**2 + t2_stats.hot_media_std**2)
    #     probabilidade_hot = calcula_probabilidade(
    #         aposta_tot_pnts, maior_pnts_do_jogo_provavel, std_teams
    #     )
    #     expectativa_hot = calcula_expectativa(probabilidade_hot, preco - 1)
    #     if expectativa_hot > EXPECTATIVE_FILTER:
    #         msg = f"$ JOGAR TOT_PNTS APOSTA: {aposta_tot_pnts}->{preco}, Expec:R${expectativa_hot}, Prob:{probabilidade_hot}, ESPERADO {maior_pnts_do_jogo_provavel}"
    #         print(msg)
    #         yield msg
    total_de_pontos_alternativas = get_total_de_pontos_alternativas(
        markets, total_de_pontos
    )

    if not total_de_pontos_alternativas:
        return None
    if type(total_de_pontos_alternativas) == list:
        rows = total_de_pontos_alternativas[0]["tableLayout"]["rows"]

    else:
        rows = total_de_pontos_alternativas["tableLayout"]["rows"]

    preco_maior_que = lambda x: x["groupSelections"][0]["selections"][0]
    for curr in rows[::-1]:
        maior_que_curr = preco_maior_que(curr)
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr < menor_pnts_do_jogo_provavel:
            preco = maior_que_curr["price"]
            std_teams = math.sqrt(t1_stats.cold_media_std**2 + t2_stats.cold_media_std**2)
            probabilidade_cold = calcula_probabilidade(
                aposta_maior_que_curr, menor_pnts_do_jogo_provavel, std_teams
            )
            expectativa_cold = calcula_expectativa(probabilidade_cold, preco - 1)
            if expectativa_cold > EXPECTATIVE_FILTER:
                msg = f"$ JOGAR TOT_PNTS(ALT) MAIOR QUE: {aposta_maior_que_curr}->{preco}, Expec:R${expectativa_cold}, Prob:{probabilidade_cold}, ESPERADO MAIOR: {menor_pnts_do_jogo_provavel}"
                print(msg)
                yield msg
                break

    preco_menor_que = lambda x: x["groupSelections"][0]["selections"][1]
    for i in range(len(rows)):
        menor_que_curr = preco_menor_que(rows[i])
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > maior_pnts_do_jogo_provavel:
            preco = menor_que_curr["price"]
            std_teams = math.sqrt(t1_stats.hot_media_std**2 + t2_stats.hot_media_std**2)
            probabilidade_hot = calcula_probabilidade(
                aposta_menor_que_curr, maior_pnts_do_jogo_provavel, std_teams
            )
            expectativa_hot = calcula_expectativa(probabilidade_hot, preco - 1)
            if expectativa_hot > EXPECTATIVE_FILTER:
                msg = f"$ JOGAR TOT_PNTS(ALT) MENOR QUE: {aposta_menor_que_curr }->{preco}, Expec:R${expectativa_hot}, Prob:{probabilidade_hot}, ESPERADO MENOR: {maior_pnts_do_jogo_provavel}"
                print(msg)
                yield msg

                break

    return None


def bet_pnts_by_team(team_stats: TeamStats, markets):
    try:
        team_tot_pnts_markets = find_by_name(
            f"Total de Pontos {team_stats.name}", markets
        )
        team_tot_pnts = team_tot_pnts_markets[0]["selections"]
        if len(team_tot_pnts_markets) > 1:
            team_tot_pnts = team_tot_pnts + team_tot_pnts_markets[1]["selections"]
        else:
            team_tot_pnts_markets = find_by_name(
                f"Total de Pontos {team_stats.name} (alternativas)", markets
            )
            if len(team_tot_pnts_markets) > 0:
                team_tot_pnts = (
                    team_tot_pnts + team_tot_pnts_markets[0]["selections"]
                )

        team_tot_pnts = list(sorted(team_tot_pnts, key=lambda x: x["handicap"]))
    except:
        return

    menor_pnts_do_jogo_provavel_team = team_stats.cold_media_mean
    maior_pnts_do_jogo_provavel_team = team_stats.hot_media_mean
    team_tot_pnts_mais_de = [o for o in team_tot_pnts if "Mais de" in o["name"]][
        ::-1
    ]
    team_tot_pnts_menos_de = [o for o in team_tot_pnts if "Menos de" in o["name"]]

    for i in range(len(team_tot_pnts_mais_de)):
        maior_que_curr = team_tot_pnts_mais_de[i]
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr < menor_pnts_do_jogo_provavel_team:
            preco = maior_que_curr["price"]
            probabilidade_cold = calcula_probabilidade(
                aposta_maior_que_curr, team_stats.cold_media_mean, team_stats.cold_std
            )
            expectativa_cold = calcula_expectativa(probabilidade_cold, preco - 1)
            if expectativa_cold > EXPECTATIVE_FILTER:
                msg = f"$ JOGAR PNTS(ALT){team_stats.name} MAIOR QUE: {aposta_maior_que_curr}->{preco}, expec:R${expectativa_cold},Prob:{probabilidade_cold}, ESPERADO MAIOR: {menor_pnts_do_jogo_provavel_team}"
                print(msg)
                yield msg
                break

    for i in range(len(team_tot_pnts_menos_de)):
        menor_que_curr = team_tot_pnts_menos_de[i]
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > maior_pnts_do_jogo_provavel_team:
            preco = menor_que_curr["price"]
            probabilidade_hot = calcula_probabilidade(
                aposta_menor_que_curr, team_stats.hot_media_mean, team_stats.hot_media_std
            )
            expectativa_hot = calcula_expectativa(probabilidade_hot, preco - 1)
            if expectativa_hot > EXPECTATIVE_FILTER:
                msg = f"$ JOGAR PNTS(ALT){team_stats.name} MENOR QUE: {aposta_menor_que_curr }->{preco}, expec:R${expectativa_hot},Prob:{probabilidade_hot}, ESPERADO MENOR: {maior_pnts_do_jogo_provavel_team}"
                print(msg)
                yield msg
                break

    return None


def get_total_de_pontos_alternativas(markets, total_de_pontos):
    total_de_pontos_alternativas = total_de_pontos
    if len(total_de_pontos) > 1:
        total_de_pontos_alternativas = total_de_pontos[1]
    else:
        total_de_pontos_alternativas = find_by_name(
            "Total de Pontos (alternativas)", markets
        )
        if len(total_de_pontos_alternativas) > 0:
            total_de_pontos_alternativas += total_de_pontos_alternativas[0]
    return total_de_pontos_alternativas


def calcula_expectativa_time(time_stats, ponto_aposta, ganho_no_jogo):
    probabilidade = calcula_probabilidade_de_pontos_do_time(ponto_aposta, time_stats)
    return calcula_expectativa(probabilidade, ganho_no_jogo)


def calcula_probabilidade_de_pontos_do_time(alvo, time_stats):
    return calcula_probabilidade(alvo, time_stats.media, time_stats.pontos.std())


def calcula_probabilidade(alvo, media, desvio_padrao):
    return round(stats.norm.cdf((alvo - media) / desvio_padrao), 2)


def calcula_expectativa(probabilidade, ganho_no_jogo, perda_no_jogo=1):
    return round(ganho_no_jogo * probabilidade - perda_no_jogo * (1 - probabilidade), 2)

In [2]:
def get_today_betano_games_urls():
    betano_games = get_betano_games()
    todays_game_url = []
    today = dt.today().date()
    for game in betano_games:
        time_obj_game = dt.fromisoformat(game["startDate"])
        date_game = time_obj_game.astimezone().date()
        if date_game == today:
            todays_game_url.append(game["url"] + "?bt=8")
        else:
            game_event = get_bet_event(game["url"])
            try:
                if game_event["liveNow"]:
                    todays_game_url.append(game["url"] + "?bt=8")
            except:
                pass

        # break
    return todays_game_url

In [3]:
import threading
import time
import time
from plyer import notification
import pygame
import ctypes  # An included library with Python install.
import winsound

# Inicializa o mixer do pygame para sons
pygame.mixer.init()


def enviar_alerta(msg, game):
    # Exibe um popup
    # notification.notify(
    #     title='Alerta',
    #     message=msg,
    #     app_name='Temporizador'
    # )
    if game in entradas_feitas:
        for sinal in entradas_feitas[game]:
            if sinal in msg:
                return False

    if ALARME_ON:
        Mbox.BoxOkCancelAsync(f"ALERTA BET {game}", msg)

        # Toca um som de alerta
        pygame.mixer.music.load("alarm.mp3")
        pygame.mixer.music.play()


class Mbox:
    OK = 1
    CANCELADO = 2

    def BoxOkCancel(title, text):
        winsound.MessageBeep(winsound.MB_OK)
        ctypes.windll.user32.MessageBoxW(0, text, title, 1)

    def Alerta(title, text=""):
        winsound.MessageBeep(winsound.MB_OK)
        funcao = lambda: ctypes.windll.user32.MessageBoxW(0, text, title, 0x1000)
        Mbox.executar_alerta_paralelo(funcao)

    def executar_alerta_paralelo(funcao_alerta, *args):
        thread_exec = ThreadSimples(funcao_alerta, *args)
        thread_exec.start()

    def BoxOkCancelAsync(title, text):
        Mbox.executar_alerta_paralelo(Mbox.BoxOkCancel, title, text)

class ThreadCustom(threading.Thread):
    def __init__(self, tempo, executar_funcao):
        super(ThreadCustom, self).__init__()
        self.parar = threading.Event()
        self.tempo = tempo
        self.executar = executar_funcao

    def run(self):
        while not self.parar.is_set():
            self.executar()
            time.sleep(self.tempo)

    def parar_execucao(self):
        print("Parando a execução...")
        self.parar.set()


def parar_thread(thread):
    thread.parar_execucao()
    thread.join()  # Aguarde até que a thread termine completamente


class ThreadSimples(threading.Thread):
    def __init__(self, funcao, *args, **kwargs):
        # If you are going to have your own constructor, make sure you call the parent constructor too!
        #        Thread.__init__(self)
        super().__init__()
        self.funcao = funcao
        self.kwargs = kwargs
        self.args = args

    def run(self):
        self.funcao(*self.args, **self.kwargs)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
betano_url_games = get_today_betano_games_urls()
zebras = get_zebra_from_today()
TEMPO_ALARME = 60 # SEGUNDOS
ALARME_ON = True
EXPECTATIVE_FILTER = 0


def pre_game_routine():
    for url in betano_url_games:
        print(">>>>")
        try:
            process_game(url)
        except Exception as e:
            print(e)


def process_game(game_url):
    try:
        event = get_bet_event(game_url)
    except:
        print(f"ERROR ON: {game_url}")
        return
    team1_name, team2_name = get_teams(event)

    if team1_name in team_stats_dict:
        t1_stats = team_stats_dict[team1_name]
    else:
        t1_stats = get_team_stats(team1_name)
        team_stats_dict[team1_name] = t1_stats

    if team2_name in team_stats_dict:
        t2_stats = team_stats_dict[team2_name]
    else:
        t2_stats = get_team_stats(team2_name)
        team_stats_dict[team2_name] = t2_stats

    if "liveNow" in event:
        datetime_info = "LIVE"
    else:
        datetime_info = dt.fromtimestamp(event["startTime"] / 1000).astimezone()

    print(f">{datetime_info} {team1_name} - {team2_name}: {game_url}")
    bet_value_expected = 3
    game_name = event["shortName"]
    markets = event["markets"]
    token_winner = "Vencedor"
    zebras_curr = zebras[zebras["name"] == game_name]
    zebras_name = zebras_curr.zebra_name.values
    t2_is_zebra = team2_name in zebras_name
    t1_is_zebra = team1_name in zebras_name
    if t1_is_zebra or t2_is_zebra:
        track_favorite = team2_name if t1_is_zebra else team1_name
        tracked_player_favorite_obj = get_bet_target(
            token_winner, track_favorite, markets
        )
        if tracked_player_favorite_obj is None:
            return
        # print(f'{token_winner}:{tracked_player_obj["name"]}:{tracked_player_obj["price"]}')
        result = has_triggered_bet(tracked_player_favorite_obj, bet_value_expected)
        if result:
            enviar_alerta(result, game_name)

    bet_on_2nd_half = can_bet_for_total_points_on_2nd_half(event,t1_stats, t2_stats)
    if bet_on_2nd_half:
        enviar_alerta(bet_on_2nd_half, game_name)

    for msg in bet_pnts_by_game(t1_stats, t2_stats, markets):
        if msg:
            enviar_alerta(msg, game_name)
    for msg in bet_pnts_by_team(t1_stats, markets):
        if msg:
            enviar_alerta(msg, game_name)

    for msg in bet_pnts_by_team(t2_stats, markets):
        if msg:
            enviar_alerta(msg, game_name)


pre_game_routine()
# process_game("https://br.betano.com/odds/boston-celtics-denver-nuggets/44146498/?bt=8")

>>>>
>LIVE Indiana Pacers - Denver Nuggets: https://br.betano.com/live/indiana-pacers-denver-nuggets/44318800/?bt=8
>>>>
>LIVE Brooklyn Nets - New York Knicks: https://br.betano.com/live/brooklyn-nets-new-york-knicks/44318810/?bt=8
$ JOGAR TOTAL DE PONTOS 2o Tempo:
215.5->1.88/esperado 198, media:223.8, expec:0.07, prob:0.37
>>>>
>LIVE New Orleans Pelicans - Utah Jazz: https://br.betano.com/live/new-orleans-pelicans-utah-jazz/44318812/?bt=8
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 256.5->2.12, Expec:R$0.25, Prob:0.59, ESPERADO MENOR: 240.61
$ JOGAR PNTS(ALT)New Orleans Pelicans MENOR QUE: 136.5->2.05, expec:R$0.29,Prob:0.63, ESPERADO MENOR: 119.87
>>>>
>LIVE Oklahoma City Thunder - Portland Trail Blazers: https://br.betano.com/live/oklahoma-city-thunder-portland-trail-blazers/44318827/?bt=8
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 235.5->2.1, Expec:R$0.05, Prob:0.5, ESPERADO MENOR: 235.01
$ JOGAR PNTS(ALT)Portland Trail Blazers MENOR QUE: 115.5->2.12, expec:R$0.14,Prob:0.54, ESPERADO MENOR: 110.58


In [12]:
entradas_feitas = {
    "New Orleans Pelicans - Utah Jazz":["JOGAR PNTS(ALT)New Orleans Pelicans MENOR QUE","TOT_PNTS(ALT) MENOR QUE","JOGAR PNTS(ALT)Utah Jazz MENOR QUE"],
    "Indiana Pacers - Denver Nuggets":["JOGAR TOT_PNTS(ALT) MAIOR QUE","JOGAR TOTAL DE PONTOS 2o Tempo","JOGAR PNTS(ALT)Denver Nuggets MENOR QUE",],
    "Oklahoma City Thunder - Portland Trail Blazers":["JOGAR TOT_PNTS(ALT) MENOR QUE", "OGAR PNTS(ALT)Oklahoma City Thunder MENOR QUE","JOGAR PNTS(ALT)Portland Trail Blazers MENOR QUE"],
    "Brooklyn Nets - New York Knicks":["JOGAR TOTAL DE PONTOS 2o Tempo","JOGAR TOT_PNTS(ALT) MAIOR QUE"],
}

23:31:37.058150
>>>>
ERROR ON: https://br.betano.com/live/indiana-pacers-denver-nuggets/44318800/?bt=8
>>>>
>LIVE Brooklyn Nets - New York Knicks: https://br.betano.com/live/brooklyn-nets-new-york-knicks/44318810/?bt=8
>>>>
>LIVE New Orleans Pelicans - Utah Jazz: https://br.betano.com/live/new-orleans-pelicans-utah-jazz/44318812/?bt=8
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 268.5->2.2, Expec:R$0.43, Prob:0.65, ESPERADO MENOR: 240.61
$ JOGAR PNTS(ALT)New Orleans Pelicans MENOR QUE: 140.5->2.05, expec:R$0.35,Prob:0.66, ESPERADO MENOR: 119.87
$ JOGAR PNTS(ALT)Utah Jazz MENOR QUE: 127.5->2.15, expec:R$0.18,Prob:0.55, ESPERADO MENOR: 120.74
>>>>
>LIVE Oklahoma City Thunder - Portland Trail Blazers: https://br.betano.com/live/oklahoma-city-thunder-portland-trail-blazers/44318827/?bt=8
$ JOGAR PNTS(ALT)Portland Trail Blazers MENOR QUE: 113.5->1.98, expec:R$0.03,Prob:0.52, ESPERADO MENOR: 110.58


In [21]:
team_stats_dict["Indiana Pacers"].cold_media_mean

23:37:46.793552
>>>>
ERROR ON: https://br.betano.com/live/indiana-pacers-denver-nuggets/44318800/?bt=8
>>>>
>LIVE Brooklyn Nets - New York Knicks: https://br.betano.com/live/brooklyn-nets-new-york-knicks/44318810/?bt=8
>>>>
>LIVE New Orleans Pelicans - Utah Jazz: https://br.betano.com/live/new-orleans-pelicans-utah-jazz/44318812/?bt=8
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 263.5->2.27, Expec:R$0.41, Prob:0.62, ESPERADO MENOR: 240.61
$ JOGAR PNTS(ALT)New Orleans Pelicans MENOR QUE: 139.5->2.12, expec:R$0.38,Prob:0.65, ESPERADO MENOR: 119.87
$ JOGAR PNTS(ALT)Utah Jazz MENOR QUE: 123.5->2.18, expec:R$0.13,Prob:0.52, ESPERADO MENOR: 120.74
>>>>
>LIVE Oklahoma City Thunder - Portland Trail Blazers: https://br.betano.com/live/oklahoma-city-thunder-portland-trail-blazers/44318827/?bt=8
$ JOGAR PNTS(ALT)Portland Trail Blazers MENOR QUE: 112.5->1.98, expec:R$0.03,Prob:0.52, ESPERADO MENOR: 110.58


In [7]:
betano_url_games = get_today_betano_games_urls()
zebras = get_zebra_from_today()


def live_routine():
    clear_output()
    print(dt.now().time())
    for url in betano_url_games:
        print(">>>>")
        process_game(url)

In [ ]:

thread = ThreadCustom(TEMPO_ALARME,live_routine)
thread.start()

23:22:18.836102
>>>>
>LIVE Indiana Pacers - Denver Nuggets: https://br.betano.com/live/indiana-pacers-denver-nuggets/44318800/?bt=8
>>>>
>LIVE Brooklyn Nets - New York Knicks: https://br.betano.com/live/brooklyn-nets-new-york-knicks/44318810/?bt=8
$ JOGAR TOTAL DE PONTOS 2o Tempo:
212.5->1.93/esperado 198, media:223.8, expec:-0.03, prob:0.33
>>>>
>LIVE New Orleans Pelicans - Utah Jazz: https://br.betano.com/live/new-orleans-pelicans-utah-jazz/44318812/?bt=8
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 262.5->2.1, Expec:R$0.3, Prob:0.62, ESPERADO MENOR: 240.61
$ JOGAR PNTS(ALT)New Orleans Pelicans MENOR QUE: 138.5->2.1, expec:R$0.37,Prob:0.65, ESPERADO MENOR: 119.87
$ JOGAR PNTS(ALT)Utah Jazz MENOR QUE: 123.5->2.0, expec:R$0.04,Prob:0.52, ESPERADO MENOR: 120.74
>>>>
>LIVE Oklahoma City Thunder - Portland Trail Blazers: https://br.betano.com/live/oklahoma-city-thunder-portland-trail-blazers/44318827/?bt=8
$ JOGAR PNTS(ALT)Portland Trail Blazers MENOR QUE: 114.5->2.05, expec:R$0.09,Prob:0.53, ESPERAD

In [ ]:
# parar_thread(thread)